In [1]:
!pip install transformers peft datasets accelerate bitsandbytes --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 39.1 MB/s eta 0:00:00


In [2]:
import torch
import math
from datasets import load_from_disk
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

In [3]:
from huggingface_hub import login

login(token="hf_tok")


In [4]:
dataset_path = "/content/drive/MyDrive/Colab Notebooks/Finetune/lm_dataset"
dataset = load_from_disk(dataset_path)
val_dataset = dataset["validation"]

In [5]:
print(f'validation set {len(val_dataset)} samples')

validation set 311 samples


Loading my base model

In [6]:
base_model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(base_model_id)
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00017-of-00019.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00018-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00019-of-00019.safetensors:   0%|          | 0.00/4.22G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Load base model + LoRA adapter

In [7]:
adapter_dir = "/content/drive/MyDrive/Colab Notebooks/Finetune/model"

base_with_adapter = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16
)
base_with_adapter = PeftModel.from_pretrained(base_with_adapter, adapter_dir)


Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

Define a function to compute perplexity

In [8]:
def calculate_perplexity(model, tokenizer, dataset, max_samples=200):
    model.eval()
    total_loss = 0
    total_tokens = 0

    for i, sample in enumerate(dataset):
        if i >= max_samples:
            break

        #Decode back from input_ids if 'text' is missing
        if "text" in sample:
            text = sample["text"]
        else:
            text = tokenizer.decode(sample["input_ids"], skip_special_tokens=True)

        inputs = tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            max_length=1024
        ).to(model.device)

        with torch.no_grad():
            outputs = model(**inputs, labels=inputs["input_ids"])
            loss = outputs.loss

        total_loss += loss.item() * inputs["input_ids"].size(1)
        total_tokens += inputs["input_ids"].size(1)

    return math.exp(total_loss / total_tokens)



In [9]:
print("Calculating perplexity)")
ppl_base = calculate_perplexity(base_model, tokenizer, val_dataset)
ppl_adapter = calculate_perplexity(base_with_adapter, tokenizer, val_dataset)

print("Base Model Perplexity:", ppl_base)
print("Base + Adapter Perplexity:", ppl_adapter)



Calculating perplexity)
Base Model Perplexity: 4.543481057286696
Base + Adapter Perplexity: 3.7041344850945546
